In [45]:
import pandas as pd
import numpy as np
import os, sys
import glob, re, time

import shapely
import folium

from collections import Counter

In [4]:
os.getcwd()

'/Volumes/LaCie/Documents/repos/particle_filter'

In [ ]:
"""
Applications of particle filters: 
1. Car positioning by map matching, as in http://www.diva-portal.org/smash/get/diva2:316556/FULLTEXT01.pdf
    Essentially same approach is described in Davidson, Collin, and Takala (2011). Application of particle filters to map-matching algorithm
    Idea also similar to Newson and Krumm (2009), though HMM is used there.
    More recent resource: Murphy, Pao, Yuen (2019). Lyft; Map matching when the map is wrong: Efficient on/off road vehicle tracking and map learning

Ideas
Rao-Blackwellization (use Kalman filter for the linear part of the dynamics model)

Initial Approach:
Use Newson and Krumm, but modify it to use particle filters instead of HMM.

So, for a given route, proceed sequentially over obs, maintaining dist of probable road segments

Assumptions (N&K):
-remove obs that are not 2*meas dist sigma from previous obs  (eliminate 39% of data in N&K)
-ignore roads 200m from obs
-zeroize very unlikely particles
"""

In [62]:
from sklearn.metrics.pairwise import haversine_distances
# Assumes (lat, long) in radians

In [ ]:
def great_circle_dist(a, b):
    """
    NOT USED
    
    Returns great-circle distance between point a and point b
    a, (lat, long) in radians
    b, (lat, long) in radians
    """
    return None

In [ ]:
class gps_transition_model:
    """
    Transition model to predict probability of next state given current state; simulate next state.
    GPS trace map matching use case. 
    
    N&K use as a proxy an exponential function of the difference in the great-circle distance 
    between previous observations and previous road points (similar route distance).
    Road points use map/ground-truth data. 
    -Ignore roads 200m from obs
    -If a calculated route would require the vehicle to exceed a speed of 50 m/s (112 miles per hour), zeroize
    
    !!! TODO: Get driving distance from route planner??? Results anticipated to significantly suffer otherwise. !!! 
    
    Other ideas: using dead reckoning.
    """
    def __init__(self, beta=3.0
#                  , prob_floor=0.05
                 , normalize=True
                ):
        """
        beta, float, parametrizes transition probability function
        prob_floor, float between 0 and 1, is probability below which road choices are given zero probability
        
        # beta = 3 in mapzen, https://www.mapzen.com/blog/data-driven-map-matching/
        """
        self.beta = beta
#         self.prob_floor = min(max(prob_floor, 0), 1)
        self.normalize = normalize
    
    def estimate_beta(self, data):
        """
        NOT USED
        
        Using rescaled median absolute deviation (MAD). 
        Ideally using ground truth data (none here).
        """ 
        self.beta = 3.0  # None
        
    def prob_transition(self, state_last2, obs_last2):
        """
        state_last2, float numpy array, (k x 4 lat long) in degrees, k candidate roads, t t-1 (pre same for all)
        obs_last2, float numpy array, (1 x 4 lat long) in degrees, t t-1
        
        returns probs of transition, (k x 1)
        """
        state_last2 = np.deg2rad(state_last2)
        obs_last2 = np.deg2rad(obs_last2)
        dist_obs = abs(haversine_distances(obs_last2[:, :2], obs_last2[0, 2:]))  # (k, 1)
        # !! TODO - Supposed to be driving distance, not haversine. N&K use a route planner. Can I use shapely???
        dist_road = np.abs(haversine_distances(state_last2[:, :2], state_last2[:, 2:]))  # (1, 1)
        diff_dist = np.abs(np.subtract(dist_road, dist_obs))  # (k, 1)

        probs = np.exp(-diff_dist / self.beta) * (1 / self.beta)
        # Normalize result?
        if self.normalize:
            probs = probs / np.sum(probs)
        return probs
    

In [ ]:
class gps_sensor_model:
    """
    Sensor model to predict likelihood of an obs given a particle.
    GPS trace map matching use case. 
    
    I don't have ground truth, so can't use ML easily, but
    N&K use as a proxy a normal distribution of great-circle distance between observation and road, with sigma 
    estimated from the data.
    Road points use map data. 
    -Zerioze low probability particles (diff in route distance of 2000 m. or more). 
    
    Other ideas: semi-supervised learning. 
    
    Thoughts: need efficient representation for road network that returns connecting roads/nodes for a given node
    Want to represent it as a graph/network. 
    """
    def __init__(self, sigma=4.07, normalize=True):
        self.sigma = sigma
        self.normalize = normalize
    
    def estimate_sigma(self, data):
        """
        NOT USED 
        
        Using Gather and Schultze, median-based. 
        Ideally using ground truth data (none here).
        
        I guess I could manually match some trips, then use that??
        """ 
        self.sigma = 4.07 # meters; I have no ground truth so using N&K's empirical estimate
#         self.sigma = None # median absolute deviation (MAD) formula adjusted 
    
    def estimate_likelihood(self, states, obs):
        """
        return probability of seeing that obs given potential road state hypotheses, (k x 1)
        
        states Particles numpy array representing road position hypotheses, (k x 2 lat long).
        obs (1 x 2 lat long)
        """
        dist_obs_roads = np.abs(haversine_distances(states, obs))  # (k x 1)
        
        probs = \
        np.exp(np.power(dist_obs_roads / self.sigma, 2) * (-0.5)) * \
        (1 / (math.sqrt(2 * math.pi) * self.sigma))
        
        # Normalize result?
        if self.normalize:
            probs = probs / np.sum(probs)
        return probs

In [2]:
class particle_filter:
    """
    Fit a particle filter to data given a transition and sensor model. 
    
    n, integer, number of particles to maintain at each step in time.
    max_road_dist_m, float, max road distance above which candidate roads aren't considered (meters). 
    prob_floor, float between 0 and 1, min probability below which candidate roads aren't considered.
    Sensor model, , is some model object we will use to predict the likelihood.
    Transition model, , is some model object we will use to predict next state.
    ## max_iter, integer, the maximum iterations used in fitting the particle filter.
    ## conv_tol, the convergence tolerance that will trigger early termination of fitting the particle filter. 
    """
    def __init__(self, n=50, max_road_dist_m=200, prob_floor=0.01, 
                 sensor_model=None, transition_model=None):
        self.n = n
        self.max_road_dist_m = max_road_dist_m
        self.prob_floor = min(max(prob_floor, 0), 1)
        self.particles = np.array([])
        self.weights = np.array([])
        
        self.sensor_model = sensor_model
        self.transition_model = transition_model
        
        self.obs = []  # Record history of observations (lat, long)
        
    def get_candidate_roads(self, obs):
        """
        Leverage shapely.
        
        Ignore roads 200m from obs.
        """
        pass
    
    def estimate_weights_per_particle(self, sampled_states, obs):
        """
        Estimate likelihood of particle given evidence, P(evidence|particle).
        Uses sensor model.
        
        Obs is single observation at time t, numpy array. 
        """
        probs = self.sensor_model.estimate_likelihood(sampled_states, obs)
        return probs
        
    def apply_transition_model(self, candidate_roads):
        """
        returns probabilities of transition, given current particles
        numpy array of floats between 0 and 1, normalized, (n * c, 1)
        
        candidate_roads numpy array of lat,long closest point to road from obs (N&K)
        
        Zeroize very small probability candidates, but don't change shape of particles.
        Then, if some particles are very unlikely, we cull them and resample from more likely particles. 
        """
        # Crossproduct for each particle, all roads
        state_last2 = np.concatenate([np.concatenate((self.particles, 
                                                      np.tile(c, (self.n, 1))), 
                                                     axis=1) 
                                      for c in candidate_roads], axis=0)  # (n * c, 4)
        obs_last2 = np.array(self.obs[-2:]).reshape((1, 4))
        probs = self.transition_model.prob_transition(state_last2, obs_last2)  # (n * c, 1)
        # Effectively, candidate roads will be possible from different original particle hypotheses
        # Zeroise small probs, without changing array shape
        probs[probs <= self.prob_floor] = 0.0
        # Re-normalize
        probs = probs / np.sum(probs)
        return probs
        
    def update_dist(self, obs):
        """
        Re-sample particles from transition model given re-estimated likelihood of existing particles.
        Weighted sample with replacement. 
        
        obs list of floats [lat, long]
        
        Note: if find no solutions, need to remove points in a signal break until HMM 'heals.'
        If break > 180 sec, separate into two trips.
        """
        # Update stored obs
        self.obs.append(obs)
        # Get candidate roads given obs
        candidate_roads = self.get_candidate_roads(obs) # c
        # Get transition probs
        trans_probs = self.apply_transition_model(candidate_roads) # (n * c, 1) first n rows for first candidate, etc.
        # Aggregate probs by candidate roads
        trans_probs_split = np.split(trans_probs, self.n, axis=0)  # (c, n)
        trans_probs_agg = np.sum(trans_probs_split, axis=1)  # (c, 1)
        # Sample new states
        sampled_states_idx = np.random.choice(range(len(candidate_roads)), 
                                              self.n, 
                                              trans_probs_agg)
        sampled_states = candidate_roads[sampled_states_idx]  # (n, 2)
        
        # Get sensor probs
        sensor_probs = self.estimate_weights_per_particle(sampled_states, obs)  # (n, 1)
        
        # Joint prob, for viterbi backtracking
        best_prior_state = np.argmax(trans_probs_split, axis=1)[sampled_states_idx]  # HERE
        joint_prob = np.multiply(
            np.multiply(sensor_probs, np.max(trans_probs_split, axis=1)[sampled_states_idx]),  # (n, 1)
            self.weights
            )
        self.weights = joint_prob
        
        # Sample new particles
        new_particles = sampled_states[np.random.choice(range(len(sampled_states)), 
                                                        self.n, 
                                                        sensor_probs)]  # (n, 2)
        self.particles = new_particles
        
        # Estimate current particle filter fit quality of hypotheses to data
        fit_quality = [np.max(self.weights), np.mean(self.weights), np.median(self.weights)]
        
        return fit_quality
        
    def fit(self, data#, max_iter=100, conv_tol=0.001
           ):
        """
        Iterate over rows in data, training particle filter. 
        Rows assumed to be sequentially ordered. 
        
        data, numpy array (num obs, data dim)
        """
        num_iter = 0
        converged = False  # Is this relevant for particle filters?
#         while num_iter < max_iter and not converged:
#             num_iter += 1
        for obs in data:
            num_iter += 1
            fit_quality = self.update_dist(obs)
            # Shouldn't I use DP/Viterbi for this?? 
            # Nearest neighbor filter, hungarian algorithm?? p. 601
            # Best at any given point in time will suffer in the beginning before particle dist has converged, ie
            # during the burn-in period.
            # So, seems can still use DP, but proceed backward in time. Wait, that is Viterbi :) 
                # Happily I have a finite state space. 
            print("On iteration %d, fit quality of MAX %3.2f, MEAN %3.2f, MEDIAN %3.2f" % 
                  (num_iter, fit_quality[0], fit_quality[1], fit_quality[2]))
        print("Done.")
        return fit_quality
    
    def viterbi(self, y_vector_arr):
        """
        Use Viterbi to get optimal path via DP - 
        happily I have a finite state space. 
        This returns the imputed GPS trace that has 
        been 'snapped' to roads.
        
        Note, Viterbi relies on the Markov property, which
        can apply here. 
        """
        
        
        
        """
        Probability of the latent variables:
        Most Likely Explanation; Joint probability of the entire sequence of hidden states 
        that generated an entire sequence of observations. 
        
        y_vector_arr numpy_array corresponding to the sequences of observations.
        y_vector_arr (num_sensors, num_sequences, max time step observed so far)
        
        Todo: How does this extend to multiple observation sequences? 
        It seems consider all sequences at each step when running forward through time. 
        Maybe, take the maximal previous state across the product of all sequences, such that
        decoded sequence may not be the viterbi solution for any individual sequence. 
        Or, use the product of the sequences (going with this).
        
        This assumes sequences are independent, and gives equal importance/weight to each sequence.
        Equal weight might not be the best idea if different sequences have different noise levels/
        accuracy. 
        Todo: flexible weights, in this function and throughout class. 
        Alternatively, first PCA/ICA transform the sequences. (Perhaps a better idea? 
        Otherwise, the estimation of relative noise levels for each sequence would be manual/
        judgmental process)
        Still nice to have a manual over-ride. 
        """
        num_obs_t = y_vector_arr.shape[2]
        if num_obs_t < 1:
            print("Error: Empty observation vector in viterbi algorithm")
            return None
        else:
            # Dynamic programming 
            # Maximal prior sequence given current evidence, conditioned on current state
            # Note: to extent to multiple sequences of different types, need to learn different B matrices!
            alpha_prev = np.multiply(self.pi, 
                                     np.prod([self.B[s, :, y_vector_arr[s, :, 0]] 
                                              for s in range(self.num_sensors)], axis=0))  # (num_states x num_seq)
#                                      self.B[:, y_vector_arr[:, 0]])  # (num_states x num_seq)
            alpha_prev = np.log(np.prod(alpha_prev, axis=1).reshape(self.num_states, 1))  # aggregate across sequences
            result = alpha_prev.copy().T  # one entry for each time t
            for h in range(1, num_obs_t):
                alpha_prev_new = np.prod([self.B[s, :, y_vector_arr[s, :, h]] 
                                          for s in range(self.num_sensors)], axis=(0, 1)).reshape(1, self.num_states)
#                 alpha_prev_new = np.prod(self.B[:, y_vector_arr[:, h]], axis=1).reshape(1, self.B.shape[0])
                alpha_prev = np.add(np.max(np.add(np.log(self.A), alpha_prev), axis=0),  # (1 x num_states)
                                    np.log(alpha_prev_new)).T
                result = np.vstack((result, alpha_prev.T))
        # Backtrack and return the most likely sequence
        most_likely_seq = np.argmax(result, axis=1).tolist()
        return most_likely_seq
        pass
    

'0.24.2'

In [15]:
## it seems i only have beijing data (10K) or sf taxi data (500)
# Read in taxi data
# Each data of different length; ideal use case for pyspark
# Note, the OSM extract basemap data has POI info as well (https://download.bbbike.org)
# also try uber h3 spatial index
trace_dir = '/Volumes/LaCie/datasets/ms_taxi/taxi_log_2008_by_id/'   # MS Taxi
basemap_dir = '/Volumes/LaCie/datasets/Beijing-shp/shape/'

trace_dir = '/Volumes/LaCie/datasets/cabspottingdata/'   # CRAWDAD cabspotting
basemap_dir = '/Volumes/LaCie/datasets/SanFrancisco-shp/shape/'

In [28]:
%%time
all_files = [f for f in os.listdir(trace_dir) if re.match(r'new_.*\.txt', f)]  # glob.glob(trace_dir + "new_*.txt")

CPU times: user 2.07 ms, sys: 906 µs, total: 2.97 ms
Wall time: 2.31 ms


In [51]:
%%time
# 20.9s cabspottingdata; can try spark or dask? or multiprocessing, joblib
trace_list = []
for file_ in all_files:
    file_df = pd.read_csv(file_, sep=" ", index_col=None, header=None, 
                          names=['lat', 'long', 'occupancy', 'time'])
    trace_list.append(file_df)

# concatenate all dfs into one
trace_df = pd.concat(trace_list, ignore_index=True)

CPU times: user 8.28 s, sys: 1.87 s, total: 10.2 s
Wall time: 14.3 s


In [ ]:
trace_df.loc[:, ["time"]] = pd.to_datetime(trace_df.time, origin="unix", unit='s')

In [59]:
"""
cabspotting:

latitude and longitude are in decimal degrees, 
occupancy shows if a cab has a fare (1 = occupied, 0 = free) and 
time is in UNIX epoch format
"""
trace_df.dtypes

lat                 float64
long                float64
occupancy             int64
time         datetime64[ns]
dtype: object

In [60]:
trace_df.head()

,lat,long,occupancy,time
0,37.75134,-122.39488,0,2008-06-10 07:58:07
1,37.75136,-122.39527,0,2008-06-10 07:57:39
2,37.75199,-122.39460,0,2008-06-10 07:55:40
3,37.75080,-122.39346,0,2008-06-10 07:54:49
4,37.75015,-122.39256,0,2008-06-10 07:50:37


In [ ]:
# Load in ...shapefiles? GeoJSON? Which format is best? For parallelization may be one thing...

In [ ]:
def preprocess_traces():
    """
    -remove obs that are not 2*meas dist sigma from previous obs  (eliminate 39% of data in N&K)
    """
    pass